# Engenharia de Atributos

In [37]:
import pandas as pd

In [38]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_7048\3999875462.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [39]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int',
    'DOMINIO_EMAIL': 'category',
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


Criando novas features (pense nelas)

Calculando tempo de casa. Indica a relação dos clientes com a empresa, isso pode influenciar na inadimplência. Clientes mais velhos de casa têm menos tendência a inadimplência.

In [40]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

In [41]:
merged_dataset['TEMPO_DE_CASA_MESES'] = merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] // 30


In [42]:
merged_dataset = merged_dataset.drop(columns='TEMPO_CADASTRO_PARA_VENCIMENTO')


Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [43]:
merged_dataset['PRAZO_PAGAMENTO_DIAS'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Mês/Ano da SAFRA_REF: capta padrões sazonais (ex: pode haver mais inadimplência no final do ano ou em meses com impostos).

In [44]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month
#merged_dataset['ANO_SAFRA'] = merged_dataset['SAFRA_REF'].dt.year ano pode não influenciar muito


Criação de **INADIMPLENCIAS_ANTERIORES**. Essa variável é um contador cumulativo do número de inadimplências passadas para cada cliente até o momento do pagamento atual. Ela captura o comportamento passado do cliente, que é um forte indicador de risco de inadimplência. Clientes reincidentes tendem a ter maior probabilidade de inadimplir novamente.

In [45]:
# Ordena por cliente e data de emissão
merged_dataset = merged_dataset.sort_values(by=['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO'])

# Cria coluna cumulativa de inadimplências
merged_dataset['INADIMPLENCIAS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .cumsum()
    .shift(fill_value=0)  # Para não contar a linha atual, evitando data leak
)


In [46]:
merged_dataset.dtypes

ID_CLIENTE                            int64
SAFRA_REF                    datetime64[ns]
DATA_EMISSAO_DOCUMENTO       datetime64[ns]
DATA_PAGAMENTO               datetime64[ns]
DATA_VENCIMENTO              datetime64[ns]
VALOR_A_PAGAR                       float64
TAXA                                float64
RENDA_MES_ANTERIOR                  float64
NO_FUNCIONARIOS                       int64
DATA_CADASTRO                datetime64[ns]
FLAG_PF                               int64
SEGMENTO_INDUSTRIAL                category
DOMINIO_EMAIL                      category
PORTE                              category
CEP_2_DIG                          category
DIAS_ATRASO                           Int64
TARGET_INADIMPLENCIA                  int64
DIAS_ADIANTAMENTO                     int64
TEMPO_DE_CASA_MESES                   int64
PRAZO_PAGAMENTO_DIAS                  int64
MES_SAFRA                             int32
INADIMPLENCIAS_ANTERIORES             int64
dtype: object

In [47]:
merged_dataset[['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'SAFRA_REF', 'RENDA_MES_ANTERIOR']]

,ID_CLIENTE,DATA_EMISSAO_DOCUMENTO,SAFRA_REF,RENDA_MES_ANTERIOR
0,8784237149961904,2018-09-04,2018-09-01,300502.0
1,8784237149961904,2018-09-06,2018-09-01,300502.0
2,8784237149961904,2018-09-09,2018-09-01,300502.0
3,8784237149961904,2018-09-11,2018-09-01,300502.0
4,8784237149961904,2018-09-17,2018-09-01,300502.0
...,...,...,...,...
72485,9206030810342980458,2021-05-16,2021-05-01,256133.0
72486,9206030810342980458,2021-05-23,2021-05-01,256133.0
72487,9206030810342980458,2021-06-16,2021-06-01,463963.0
72488,9206030810342980458,2021-06-18,2021-06-01,463963.0


Pré-processamento dos dados para o modelo 

Colunas a serem removidas:

- ID_CLIENTE: funciona como um identificador, não possui informação útil para predição.
- DATA_EMISSAO_DOCUMENTO, DATA_PAGAMENTO, DATA_VENCIMENTO, DATA_CADASTRO, SAFRA_REF: São datas brutas. Foram transformadas em variáveis derivadas.
- DIAS_ATRASO, DIAS_ADIANTAMENTO: colunas que só são conhecidas quando se sabe sobre a variável target. São removidas para evitar datal leak e consequentemente overfitting


Colunas categóricas a serem tratadas: 
- SEGMENTO_INDUSTRIAL, PORTE, DOMINIO_EMAIL: aplicão One-Hot Encoding pois são váriáveis categóricas de baixa cardinalidade




# LIDAR COM A ALTA CARDINALIDADE DE CEP_2_DIG POSTERIORMENTE


Aplicando o pré-processamento 

In [48]:
# Eliminar colunas irrelevantes
cols_drop = [
    'ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
    'DATA_VENCIMENTO', 'DATA_CADASTRO', 'SAFRA_REF', 'CEP_2_DIG',
    'DIAS_ATRASO', 'DIAS_ADIANTAMENTO', # Colunas que só são conhecidas depois do treinamento. São removidas para evitar data leakage
    #'INADIMPLENCIAS_ANTERIORES', # verificar se essa variável é meu data leakage
]

df = merged_dataset.drop(columns=cols_drop)


# Aplicando One-Hot Encoding em variáveis categóricas de baixa cardinalidade
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL', 'PORTE', 'DOMINIO_EMAIL'], drop_first=True)

X = df.drop(columns='TARGET_INADIMPLENCIA')
y = df['TARGET_INADIMPLENCIA']

In [49]:
X

,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,FLAG_PF,TEMPO_DE_CASA_MESES,PRAZO_PAGAMENTO_DIAS,MES_SAFRA,INADIMPLENCIAS_ANTERIORES,SEGMENTO_INDUSTRIAL_Indústria,SEGMENTO_INDUSTRIAL_Serviços,PORTE_MEDIO,PORTE_PEQUENO,DOMINIO_EMAIL_HOTMAIL,DOMINIO_EMAIL_OUTROS,DOMINIO_EMAIL_YAHOO
0,59610.76,5.99,300502.0,107,0,92,20,9,0,False,False,False,True,True,False,False
1,39398.06,5.99,300502.0,107,0,92,18,9,0,False,False,False,True,True,False,False
2,55416.75,5.99,300502.0,107,0,92,16,9,0,False,False,False,True,True,False,False
3,11751.35,5.99,300502.0,107,0,92,16,9,0,False,False,False,True,True,False,False
4,35985.00,5.99,300502.0,107,0,92,16,9,0,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72485,107321.36,11.99,256133.0,109,0,192,16,5,2,False,True,True,False,True,False,False
72486,38372.91,5.99,256133.0,109,0,192,16,5,2,False,True,True,False,True,False,False
72487,43030.50,5.99,463963.0,105,0,193,16,6,2,False,True,True,False,True,False,False
72488,107318.81,5.99,463963.0,105,0,193,17,6,2,False,True,True,False,True,False,False


In [50]:
y

0        0
1        0
2        0
3        0
4        0
        ..
72485    0
72486    0
72487    0
72488    0
72489    0
Name: TARGET_INADIMPLENCIA, Length: 72490, dtype: int64

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# === 1. PREPARAÇÃO DOS DADOS ===
# Supondo que você já tenha seu dataframe pronto com as variáveis transformadas
# e a variável target separada como `TARGET_INADIMPLENCIA`

# Exemplo:
X = df.drop(columns=['TARGET_INADIMPLENCIA'])
y = df['TARGET_INADIMPLENCIA']

# Divisão treino/teste (estratificada por causa do desbalanceamento)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === 2. DEFINIÇÃO DO MODELO BASE ===
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),  # Contornando o desbalanceamento
)

# === 3. GRID DE PARÂMETROS ===
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# === 4. VALIDAÇÃO CRUZADA ===
# Preservando a quantidade de elementos em cada classe
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

# === 5. TREINAMENTO ===
grid.fit(X_train, y_train)

# === 6. RESULTADOS ===
print("Melhores hiperparâmetros:")
print(grid.best_params_)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores hiperparâmetros:
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     13432
           1       0.61      0.79      0.69      1066

    accuracy                           0.95     14498
   macro avg       0.80      0.88      0.83     14498
weighted avg       0.96      0.95      0.95     14498


Confusion Matrix:
[[12897   535]
 [  219   847]]

ROC AUC Score:
0.9513068816089271


In [52]:

#X.to_csv('../data/processed/X.csv', index=False)

In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# === 1. Preparação dos dados ===
# Substitua pelo seu DataFrame real
#X = df.drop(columns=["TARGET_INADIMPLENCIA"])
#y = df["TARGET_INADIMPLENCIA"]

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# === 2. Modelos e grids ===
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

modelos_parametros = {
    "XGBoost": (
        XGBClassifier(
            use_label_encoder=False,
            objective="binary:logistic",
            eval_metric="logloss",
            #scale_pos_weight=scale_pos_weight, # ignorado pois já usamos SMOTE
            random_state=42
        ),
        {
            "clf__n_estimators": [100, 200],
            "clf__max_depth": [3, 5, 7],
            "clf__learning_rate": [0.01, 0.1],
            "clf__subsample": [0.8],
            "clf__colsample_bytree": [0.8]
        }
    ),
    "RandomForest": (
        RandomForestClassifier(random_state=42),
        {
            "clf__n_estimators": [100, 200],
            "clf__max_depth": [None, 10],
        }
    ),
    "LogisticRegression": (
        LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42),
        {
            "clf__C": [0.1, 1.0, 10.0]
        }
    ),
    "LightGBM": (
        LGBMClassifier(random_state=42),
        {
            "clf__n_estimators": [100, 200],
            "clf__num_leaves": [31, 50],
            "clf__learning_rate": [0.01, 0.1]
        }
    )
}

# === 3. Validação cruzada e execução ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = {}

for nome, (modelo, grid_params) in modelos_parametros.items():
    print(f"\nTreinando: {nome}")
    pipeline = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("smote", SMOTE(random_state=42)),
        ("clf", modelo)
    ])

    grid = GridSearchCV(
        pipeline,
        param_grid=grid_params,
        cv=cv,
        scoring="roc_auc",
        verbose=1,
        n_jobs=-1
    )

    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    resultados[nome] = {
        "Melhores parâmetros": grid.best_params_,
        "ROC AUC (validação)": grid.best_score_,
        "ROC AUC (teste)": roc_auc_score(y_test, y_proba),
        "Classification Report": classification_report(y_test, y_pred, output_dict=True),
        "Confusion Matrix": confusion_matrix(y_test, y_pred).tolist()
    }

# === 4. Resultados Finais ===
for nome, res in resultados.items():
    print(f"\n=== {nome} ===")
    print("Melhores parâmetros:", res["Melhores parâmetros"])
    print("ROC AUC (validação):", round(res["ROC AUC (validação)"], 4))
    print("ROC AUC (teste):", round(res["ROC AUC (teste)"], 4))
    print("Classification Report:")
    print(pd.DataFrame(res["Classification Report"]).T)
    print("Confusion Matrix:")
    print(np.array(res["Confusion Matrix"]))



Treinando: XGBoost
Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\Users\Matheus\miniconda3\envs\py310\lib\site-packages\xgboost\training.py:183: UserWarning: [20:37:21] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Treinando: RandomForest
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Treinando: LogisticRegression
Fitting 5 folds for each of 3 candidates, totalling 15 fits

Treinando: LightGBM
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Number of positive: 53726, number of negative: 53726
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0,004489 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3096
[LightGBM] [Info] Number of data points in the train set: 107452, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,500000 -> initscore=0,000000

=== XGBoost ===
Melhores parâmetros: {'clf__colsample_bytree': 0.8, 'clf__learning_rate': 0.1, 'clf__max_depth': 7, 'clf__n_estimators': 200, 'clf__subsample': 0.8}
ROC AUC (validação): 0.949
ROC AUC (teste): 0.9507
Classification Report:
              precision    recall  f1-score      support
0     

c:\Users\Matheus\miniconda3\envs\py310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Matheus\miniconda3\envs\py310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [54]:
X_test

,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,FLAG_PF,TEMPO_DE_CASA_MESES,PRAZO_PAGAMENTO_DIAS,MES_SAFRA,INADIMPLENCIAS_ANTERIORES,SEGMENTO_INDUSTRIAL_Indústria,SEGMENTO_INDUSTRIAL_Serviços,PORTE_MEDIO,PORTE_PEQUENO,DOMINIO_EMAIL_HOTMAIL,DOMINIO_EMAIL_OUTROS,DOMINIO_EMAIL_YAHOO
17544,35533.00,5.99,384638.0,116,0,55,16,3,0,False,False,False,True,False,True,False
55267,195671.16,6.99,215668.0,107,0,254,16,6,2,True,False,True,False,False,False,True
305,74683.50,5.99,236510.0,110,0,75,18,4,1,False,True,False,True,False,False,False
32948,78513.69,5.99,318541.0,117,0,227,27,3,6,True,False,False,False,False,False,False
37649,72604.10,6.99,690600.0,130,0,52,18,2,1,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67173,34262.40,6.99,261759.0,114,0,92,16,5,0,False,False,False,True,False,True,False
28345,14951.00,11.99,217636.0,107,0,7,46,8,0,False,False,False,False,False,False,True
18836,21351.12,4.99,128086.0,112,0,192,16,6,2,False,False,False,False,False,False,True
26386,41613.10,6.99,477140.0,97,0,96,17,12,0,True,False,False,False,False,False,True
